# Tool Usage Examples

This notebook demonstrates how to use the OpenAPI to Pydantic-AI tools converter with practical examples from both the AI Knowledge API and Ally Config API.

In [1]:
# Import required libraries
from meta_ally.lib.auth_manager import AuthManager

from meta_ally.lib.openapi_to_tools import OpenAPIToolsLoader

# We'll use these for our examples
print("✅ Imports loaded successfully!")

✅ Imports loaded successfully!


In [2]:
# Initialize AuthManager for authentication
print("Initializing AuthManager...")

auth_manager = AuthManager(
    keycloak_url="https://keycloak.acc.iam-services.aws.inform-cloud.io/",
    realm_name="inform-ai",
    client_id="ally-portal-frontend-dev"
)

print("✅ AuthManager initialized!")

Initializing AuthManager...
✅ AuthManager initialized!


In [ ]:
# Load tools from Ally Config API
print("Loading Ally Config API tools...")

ally_config_loader = OpenAPIToolsLoader(
    openapi_url="https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json",
    models_filename="ally_config_api_models.py",
    regenerate_models=True  # Generate the models file
)

print("✅ Ally Config loader initialized.")
ally_config_tools = ally_config_loader.load_tools()

print(f"✅ Loaded {len(ally_config_tools)} Ally Config tools!")
print("\nFirst 5 tools:")
for i, tool in enumerate(ally_config_tools[:5], 1):
    print(f"  {i}. {tool.name}")

Loading Ally Config API tools...
✅ Ally Config loader initialized.
Generating models file 'ally_config_api_models.py' from https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json...
Generating models file 'ally_config_api_models.py' from https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json...
Successfully generated 'ally_config_api_models.py'
Created tool: get_portal_config [GET /api/info/config]
Created tool: list_models [GET /api/info/models]
Created tool: list_scopes [GET /api/info/scopes]
Created tool: list_copilots [GET /api/copilots]
Created tool: create_copilot [POST /api/copilots]
Created tool: delete_copilot [DELETE /api/copilots]
Created tool: get_copilot_metadata [GET /api/copilots/metadata]
Created tool: update_copilot_metadata [PUT /api/copilots/metadata]
Created tool: get_copilot_config [GET /api/copilots/config]
Created tool: update_copilot_config [POST /api/copilots/config]
Created tool: validate_copilot_config [POST /api/copilots/config/valid

In [4]:
print("🚀 Practical tool execution pattern:\n")


async def execute_tool_safely(tool, **kwargs):
    """
    Safely execute a tool with proper error handling.

    Returns:
        The result of the tool execution, or None if an error occurs.
    """
    try:
        # Create dependencies from the loader with auth_manager
        dependencies = ally_config_loader.create_dependencies(auth_manager=auth_manager)

        # Create a proper RunContext with dependencies
        class MockRunContext:
            def __init__(self, deps):
                self.deps = deps

        ctx = MockRunContext(dependencies)

        print(f"🔄 Attempting to call: {tool.name}")
        result = await tool.function(ctx, **kwargs)
        print(f"✅ Success! Result type: {type(result)!r}")
        return result

    except Exception as e:
        print(f"⚠️ Error calling {tool.name}: {e!s}")
        return None

🚀 Practical tool execution pattern:



In [5]:
get_available_ai_models_tool = ally_config_loader.get_tool_by_operation_id("list_models")
if get_available_ai_models_tool:
    print(await execute_tool_safely(get_available_ai_models_tool))
    print(get_available_ai_models_tool.tool_def)
else:
    print("❌ Tool 'list_models' not found")

🔄 Attempting to call: list_models
Obtaining authorization token...


'\\wsl.localhost\Ubuntu-24.04\home\jschillb\meta_ally\examples'
CMD.EXE was started with the above path as the current directory.
UNC paths are not supported.  Defaulting to Windows directory.


Token obtained, expires at: 1765453992.114983     
Token obtained, expires at: 1765453992.114983     
✅ Success! Result type: <class 'list'>
[{'label': 'gpt-5-nano', 'name': 'GPT 5 nano', 'description': 'GPT 5 nano (€)\nThe gpt-5 series is the newest generation of models from OpenAI. gpt-5-nano supports both text and vision at extremely low cost and latency, making it ideal for high-volume applications.\n', 'enabled': True}, {'label': 'gpt-5-mini', 'name': 'GPT 5 mini', 'description': 'GPT 5 mini (€€)\nThe gpt-5 series is the newest generation of models from OpenAI. gpt-5-mini provides enhanced performance with low cost and latency, supporting both text and vision capabilities.\n', 'enabled': True}, {'label': 'gpt-4.1-nano', 'name': 'GPT 4.1 nano', 'description': 'GPT 4.1 nano (€)\nThe gpt-4.1 series is the latest iteration of the gpt-4o model family. gpt-4.1-nano supports both text and vision provides at even lower cost and latency compared to the gpt-4o and gpt-4o-mini models.\nhttps

In [6]:
create_endpoint_tool = ally_config_loader.get_tool_by_operation_id("create_copilot")
if create_endpoint_tool:
    print(create_endpoint_tool.tool_def)
else:
    print("❌ Tool 'create_copilot' not found")

ToolDefinition(name='create_copilot', parameters_json_schema={'type': 'object', 'properties': {'endpoint': {'type': 'string', 'description': 'The endpoint identifier for the Copilot (format: /SCOPE/NAME)'}, 'endpoint_attributes': {'properties': {'dep_name': {'type': 'string', 'title': 'Dep Name', 'description': 'Model of the endpoint'}, 'instructions': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Instructions', 'description': 'Instructions for the endpoint'}, 'default_message': {'type': 'string', 'title': 'Default Message', 'description': 'Default message for the endpoint'}}, 'type': 'object', 'required': ['dep_name', 'default_message'], 'title': 'EndpointAttributes', 'description': 'Attributes for the new endpoint including dep_name, instructions, and default_message'}, 'endpoint_metadata': {'properties': {'display_name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Display Name', 'description': 'Human-readable name of the endpoint'}, 'description': {'any

In [ ]:
# Load tools from AI Knowledge API
print("Loading AI Knowledge API tools...")

ai_knowledge_loader = OpenAPIToolsLoader(
    openapi_url="https://backend-api.dev.ai-knowledge.aws.inform-cloud.io/openapi.json",
    models_filename="ai_knowledge_api_models.py",
    regenerate_models=True  # Generate the models file
)

print("✅ AI Knowledge loader initialized.")
ai_knowledge_tools = ai_knowledge_loader.load_tools()

print(f"✅ Loaded {len(ai_knowledge_tools)} AI Knowledge tools!")
print("\nFirst 5 tools:")
for i, tool in enumerate(ai_knowledge_tools[:5], 1):
    print(f"  {i}. {tool.name}")

Loading AI Knowledge API tools...
✅ AI Knowledge loader initialized.


Generating models file 'ai_knowledge_api_models.py' from https://backend-api.dev.ai-knowledge.aws.inform-cloud.io/openapi.json...
Successfully generated 'ai_knowledge_api_models.py'
Created tool: list_collections [GET /collections/]
Created tool: create_collection [POST /collections/]
Created tool: get_collection_configuration [GET /collections/{collection_id}]
Created tool: update_collection [PUT /collections/{collection_id}]
Created tool: delete_collection [DELETE /collections/{collection_id}]
Created tool: index_collection [POST /collections/{collection_id}/index]
Created tool: query [POST /collections/{collection_id}/query]
Created tool: get_fragment_counts [GET /collections/{collection_id}/fragments/counts]
Created tool: get_collection_usage [GET /collections/{collection_id}/usage]
Created tool: list_sources [GET /sources/]
Created tool: create_source [POST /sources/]
Created tool: get_source_configuration [GET /sources/{source_id}]
Created tool: update_source [PUT /sources/{sourc

In [8]:
create_collection_tool = ai_knowledge_loader.get_tool_by_operation_id("create_collection")
if create_collection_tool:
    print(create_collection_tool.tool_def)
else:
    print("❌ Tool 'create_collection' not found")

ToolDefinition(name='create_collection', parameters_json_schema={'type': 'object', 'properties': {'collectionId': {'type': 'string', 'maxLength': 62, 'minLength': 8, 'pattern': '^[a-zA-Z_][a-zA-Z0-9_]*$', 'title': 'Collectionid', 'description': 'A unique identifier for the collection.'}, 'description': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Description', 'description': 'Description of the collection.'}, 'projectNumber': {'type': 'string', 'pattern': '^(00000|[1-9]\\d{4}|1\\d{5})$', 'title': 'Projectnumber', 'description': 'Project number related to the collection. This number is used to derive the business department (GB) and to allocate costs. Note: Changes to this field take effect on cost allocation at the next day (00:00 UTC).', 'default': '00000'}, 'sources': {'items': {'type': 'string', 'maxLength': 255, 'minLength': 0, 'pattern': '^[a-zA-Z0-9-_]+$', 'description': 'A list of source identifiers associated with the collection.'}, 'type': 'array', 'maxItems': 3

In [9]:
create_source_tool = ai_knowledge_loader.get_tool_by_operation_id("create_source")
if create_source_tool:
    print(create_source_tool.tool_def)
else:
    print("❌ Tool 'create_source' not found")

ToolDefinition(name='create_source', parameters_json_schema={'type': 'object', 'properties': {'sourceId': {'type': 'string', 'maxLength': 255, 'minLength': 8, 'pattern': '^[a-zA-Z0-9-_]+$', 'title': 'Sourceid', 'description': 'A unique identifier for the data source.'}, 'description': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Description', 'description': 'Description of the data source.'}, 'public': {'type': 'boolean', 'title': 'Public', 'description': 'Flag indicating if the source is publicly accessible.', 'default': False}, 'properties': {'oneOf': [{'properties': {'prefix': {'items': {'type': 'string'}, 'type': 'array', 'maxItems': 32, 'minItems': 0, 'title': 'Prefix', 'description': 'A list of prefixes to filter objects in the source.', 'default': []}, 'suffix': {'items': {'type': 'string'}, 'type': 'array', 'maxItems': 32, 'minItems': 0, 'title': 'Suffix', 'description': 'A list of suffixes to filter objects in the source.', 'default': []}, 'prefixIgnore': {'item

In [10]:
update_collection_tool = ai_knowledge_loader.get_tool_by_operation_id("update_collection")
if update_collection_tool:
    print(update_collection_tool.tool_def.parameters_json_schema)
else:
    print("❌ Tool 'update_collection' not found")

{'type': 'object', 'properties': {'collection_id': {'type': 'string', 'description': ''}, 'collectionId': {'type': 'string', 'maxLength': 62, 'minLength': 8, 'pattern': '^[a-zA-Z_][a-zA-Z0-9_]*$', 'title': 'Collectionid', 'description': 'A unique identifier for the collection.'}, 'description': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'title': 'Description', 'description': 'Description of the collection.'}, 'projectNumber': {'type': 'string', 'pattern': '^(00000|[1-9]\\d{4}|1\\d{5})$', 'title': 'Projectnumber', 'description': 'Project number related to the collection. This number is used to derive the business department (GB) and to allocate costs. Note: Changes to this field take effect on cost allocation at the next day (00:00 UTC).', 'default': '00000'}, 'sources': {'items': {'type': 'string', 'maxLength': 255, 'minLength': 0, 'pattern': '^[a-zA-Z0-9-_]+$', 'description': 'A list of source identifiers associated with the collection.'}, 'type': 'array', 'maxItems': 32, 'minI

In [11]:
async def execute_ai_knowledge_tool_safely(tool, **kwargs):
    """
    Safely execute an AI Knowledge tool with proper error handling.

    Returns:
        The result of the tool execution, or None if an error occurs.
    """
    try:
        # Create dependencies from the AI Knowledge loader with auth_manager
        dependencies = ai_knowledge_loader.create_dependencies(auth_manager=auth_manager)

        # Create a proper RunContext with dependencies
        class MockRunContext:
            def __init__(self, deps):
                self.deps = deps

        ctx = MockRunContext(dependencies)

        print(f"🔄 Attempting to call: {tool.name}")
        result = await tool.function(ctx, **kwargs)
        print(f"✅ Success! Result type: {type(result)!r}")
        return result

    except Exception as e:
        print(f"⚠️ Error calling {tool.name}: {e!s}")
        return None


# Execute AI Knowledge tool
list_sources_tool = ai_knowledge_loader.get_tool_by_operation_id("list_sources")
await execute_ai_knowledge_tool_safely(list_sources_tool)

🔄 Attempting to call: list_sources


✅ Success! Result type: <class 'dict'>


{'Count': 20,
 'ScannedCount': 96,
 'TotalCount': 45,
 'Items': [{'sourceId': 'addone-infopapers-sharepoint',
   'description': 'SharePoint-Ordner mit AddOne-InfoPapers für Sales und Produktinfos',
   'public': False,
   'sourceType': 'sharepoint',
   'accessDenied': False},
  {'sourceId': 'addone-sharepoint',
   'description': 'SharePoint Ordner mit AddOne InfoPapers',
   'public': False,
   'sourceType': 'sharepoint',
   'accessDenied': False},
  {'sourceId': 'addone_infopapers',
   'description': 'SharePoint Ordner mit AddOne-InfoPapers Broschüren',
   'public': False,
   'sourceType': 'sharepoint',
   'accessDenied': False},
  {'sourceId': 'addone_infopapers_sharepoint_gb10',
   'description': 'SharePoint-Ordner mit AddOne-InfoPapers für Sales Copilot GB10',
   'public': False,
   'sourceType': 'sharepoint',
   'accessDenied': False},
  {'sourceId': 'addone_infopapers_sharepoint_gb10_80000',
   'description': 'SharePoint-Ordner mit AddOne-InfoPapers für Sales Copilot GB10/80000',
 

# Using the ToolGroupManager

Now let's demonstrate how to use the ToolGroupManager which provides a more organized way to work with API tools and includes convenient methods for executing tools by operation ID.

In [12]:
# Import and initialize the ToolGroupManager
from meta_ally.util.tool_group_manager import ToolGroupManager

print("Setting up ToolGroupManager...")
tool_manager = ToolGroupManager(auth_manager=auth_manager)

# Load both API tools
print("Loading AI Knowledge tools...")
tool_manager.load_ai_knowledge_tools(regenerate_models=False)  # Use existing models

print("Loading Ally Config tools...")
tool_manager.load_ally_config_tools(regenerate_models=False)  # Use existing models

print("✅ ToolGroupManager setup complete!")

Setting up ToolGroupManager...
Loading AI Knowledge tools...


Created tool: ai_knowledge_list_collections [GET /collections/]
Created tool: ai_knowledge_create_collection [POST /collections/]
Created tool: ai_knowledge_get_collection_configuration [GET /collections/{collection_id}]
Created tool: ai_knowledge_update_collection [PUT /collections/{collection_id}]
Created tool: ai_knowledge_delete_collection [DELETE /collections/{collection_id}]
Created tool: ai_knowledge_index_collection [POST /collections/{collection_id}/index]
Created tool: ai_knowledge_query [POST /collections/{collection_id}/query]
Created tool: ai_knowledge_get_fragment_counts [GET /collections/{collection_id}/fragments/counts]
Created tool: ai_knowledge_get_collection_usage [GET /collections/{collection_id}/usage]
Created tool: ai_knowledge_list_sources [GET /sources/]
Created tool: ai_knowledge_create_source [POST /sources/]
Created tool: ai_knowledge_get_source_configuration [GET /sources/{source_id}]
Created tool: ai_knowledge_update_source [PUT /sources/{source_id}]
Create

In [13]:
# Example 1: Get a tool by operation ID
print("🔍 Finding tool by operation ID...")
tool = tool_manager.get_tool_by_operation_id("get_available_AI_models_api_getAvailableAIModels_post")
if tool:
    print(f"✅ Found tool: {tool.name}")
else:
    print("❌ Tool not found")

# Example 2: Execute a tool safely using the convenience method
print("\n🚀 Executing tool using execute_tool_safely method...")
result = await tool_manager.execute_tool_safely("get_available_AI_models_api_getAvailableAIModels_post")
if result:
    print(f"📊 Tool execution successful! Result has {len(result) if isinstance(result, list) else 'data'}")
else:
    print("❌ Tool execution failed or returned None")

🔍 Finding tool by operation ID...
❌ Tool not found

🚀 Executing tool using execute_tool_safely method...


ValueError: Tool 'get_available_AI_models_api_getAvailableAIModels_post' not found in loaded tools

In [ ]:
# Example 3: Execute an AI Knowledge tool
print("🧠 Trying AI Knowledge tool...")
ai_result = await tool_manager.execute_tool_safely("list_sources")
if ai_result:
    print("✅ AI Knowledge tool successful!")
else:
    print("(i) AI Knowledge tool may require additional parameters or authentication")

# Example 4: Show available tool groups information
print("\n📋 Available tool groups:")
groups_info = tool_manager.get_available_groups()

print(f"AI Knowledge groups: {list(groups_info['ai_knowledge_groups'].keys())}")
print(f"Ally Config groups: {list(groups_info['ally_config_groups'].keys())}")

# Show specific tools in a group (example: sources)
if 'sources' in groups_info['ai_knowledge_groups']:
    sources_tools = groups_info['ai_knowledge_groups']['sources']
    print(f"\nSource management tools: {sources_tools}")

🧠 Trying AI Knowledge tool...
🔄 Attempting to call: ai_knowledge_list_sources
✅ Success! Result type: <class 'dict'>
✅ AI Knowledge tool successful!

📋 Available tool groups:
AI Knowledge groups: ['sources', 'documents', 'search', 'retrieval', 'processing', 'metadata', 'collections', 'indexing', 'permissions', 'admin']
Ally Config groups: ['portal_info', 'copilot', 'evaluation', 'analytics', 'permissions', 'file_operations']

Source management tools: ['ai_knowledge_list_sources', 'ai_knowledge_create_source', 'ai_knowledge_get_source_configuration', 'ai_knowledge_update_source', 'ai_knowledge_delete_source']
✅ Success! Result type: <class 'dict'>
✅ AI Knowledge tool successful!

📋 Available tool groups:
AI Knowledge groups: ['sources', 'documents', 'search', 'retrieval', 'processing', 'metadata', 'collections', 'indexing', 'permissions', 'admin']
Ally Config groups: ['portal_info', 'copilot', 'evaluation', 'analytics', 'permissions', 'file_operations']

Source management tools: ['ai_kn